### Cleaning

#### 1. LLM으로 1차 분류
- train.csv에서 자연스러운 문장과 부자연스러운 문장을 gemini로 분류
- -> clean.csv, noise.csv 생성

In [ ]:
"""
gemini 1.5 flash LLM을 이용하여 train.csv의 text가 자연스러운지 부자연스러운지 분류하고,
자연스러운 문장은 clean.csv에, 부자연스러운 문장은 noise.csv에 저장하는 코드입니다.
"""
### LIBRARY ###
import pandas as pd
import os
import time
import re
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
import glob
###############

### SETTING ###
API_KEY = ''
INPUT_FILE = 'data/train.csv'
###############

# API 키 설정
os.environ["GOOGLE_API_KEY"] = API_KEY

# 데이터 로드: 모든 행 읽기
data_file = INPUT_FILE
try:
    data = pd.read_csv(data_file)
    print(f"총 {len(data)}개의 데이터를 로드했습니다.")
except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다: {data_file}")
    exit(1)
except Exception as e:
    print(f"데이터 로드 중 오류 발생: {e}")
    exit(1)

# 모델 초기화
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

# 프롬프트 템플릿 정의
prompt_template = PromptTemplate(
    input_variables=["batch"],
    template="""
    다음은 뉴스 제목 형식의 텍스트 10개입니다:

    {batch}

    각 텍스트가 자연스러운지 부자연스러운지 판단해 주세요. 

    자연스러운 문장의 특징:
    문법적으로 올바르고 의미가 명확합니다.
    일반적인 뉴스 기사에서 사용되는 표현과 어휘를 사용합니다.
    특별한 이상 문자나 오탈자가 없습니다.
    문장의 대부분이 한글로 이루어져 있습니다.
    
    부자연스러운 문장의 특징:
    문법 오류가 있거나 의미가 모호합니다.
    이상한 문자, 기호, 알 수 없는 약어 등이 포함되어 있습니다.
    읽기 어렵거나 이해하기 힘든 표현이 있습니다.

    반드시 각 번호에 해당하는 텍스트의 분류 결과를 'natural' 또는 'unnatural'으로만 대답해 주세요.
    응답은 번호 순서대로 한 줄에 하나씩 작성해 주세요.
    """
)

# 결과를 저장할 리스트 초기화
clean_results = []
noise_results = []

# 배치 크기 설정
batch_size = 10
total_rows = len(data)

# 출력 파일명 설정
clean_output_file = '1_clean.csv'
noise_output_file = '2_noise.csv'

# 기존 결과 파일이 존재하면 삭제하여 새로 생성
if os.path.exists(clean_output_file):
    os.remove(clean_output_file)
    print(f"{clean_output_file} 파일을 삭제하고 새로 생성합니다.")
if os.path.exists(noise_output_file):
    os.remove(noise_output_file)
    print(f"{noise_output_file} 파일을 삭제하고 새로 생성합니다.")

# 배치 처리
for start_idx in range(0, total_rows, batch_size):
    end_idx = min(start_idx + batch_size, total_rows)
    batch = data.iloc[start_idx:end_idx]

    batch_texts = []
    batch_ids = []
    for i, (index, row) in enumerate(batch.iterrows(), 1):
        id = row['ID']
        text = row['text']
        target = row['target']
        batch_texts.append(f"{i}. ID: {id}, Text: {text}")
        batch_ids.append(id)

    batch_prompt = "\n".join(batch_texts)

    # 프롬프트 생성
    prompt = prompt_template.format(batch=batch_prompt)

    success = False
    retry_count = 0
    max_retries = 5

    while not success and retry_count < max_retries:
        try:
            # 모델 예측
            response = model.invoke(prompt)

            # AIMessage 객체에서 실제 텍스트 추출
            response_text = response.content.strip()
            classifications = response_text.split('\n')

            if len(classifications) != len(batch_ids):
                raise ValueError("응답된 분류 개수가 입력된 텍스트 개수와 일치하지 않습니다.")

            for idx, classification in enumerate(classifications):
                id = batch_ids[idx]
                text = batch.iloc[idx]['text']
                target = batch.iloc[idx]['target']

                # 번호와 점을 제거하여 실제 분류 결과 추출
                match = re.match(r'\d+\.\s*(.+)', classification)
                if match:
                    classification = match.group(1).strip()
                else:
                    classification = classification.strip()

                if classification == 'natural':
                    clean_results.append({
                        "ID": id,
                        "text": text,
                        "target": target
                    })
                elif classification == 'unnatural':
                    noise_results.append({
                        "ID": id,
                        "text": text,
                        "target": target
                    })
                else:
                    # 예기치 않은 응답은 부자연스러운 문장으로 처리
                    noise_results.append({
                        "ID": id,
                        "text": text,
                        "target": target
                    })

                print(f"ID {id} 처리 완료: {classification}")

            success = True

            time.sleep(2)  # request 사이에 지연

        except Exception as e:
            retry_count += 1
            print(f"배치 {start_idx + 1}-{end_idx} 처리 중 오류 발생: {e}")
            if 'ResourceExhausted' in str(e):
                wait_time = 30  # 쿼터 초과 시 30초 대기
                print(f"{wait_time}초 후에 재시도합니다.")
                time.sleep(wait_time)
            else:
                wait_time = 10  # 기타 오류 시 10초 대기
                print(f"{wait_time}초 후에 재시도합니다.")
                time.sleep(wait_time)

    if not success:
        print(f"배치 {start_idx + 1}-{end_idx} 처리를 건너뜁니다. 부자연스러운 문장으로 저장합니다.")
        for id in batch_ids:
            text = data.loc[data['ID'] == id, 'text'].values[0]
            target = data.loc[data['ID'] == id, 'target'].values[0]
            noise_results.append({
                "ID": id,
                "text": text,
                "target": target
            })
            print(f"ID {id} 처리를 실패하여 부자연스러운 문장으로 저장했습니다.")

    # clean.csv 저장
    if clean_results:
        clean_df = pd.DataFrame(clean_results)
        if start_idx == 0:
            clean_df.to_csv(clean_output_file, index=False, encoding='utf-8-sig')
        else:
            clean_df.to_csv(clean_output_file, mode='a', index=False, header=False, encoding='utf-8-sig')
        print(f"Clean 데이터 저장 완료: {len(clean_results)}개")
        clean_results = []
    else:
        print("Clean 결과가 없습니다. clean.csv를 저장하지 않습니다.")

    # noise.csv 저장
    if noise_results:
        noise_df = pd.DataFrame(noise_results)
        if start_idx == 0:
            noise_df.to_csv(noise_output_file, index=False, encoding='utf-8-sig')
        else:
            noise_df.to_csv(noise_output_file, mode='a', index=False, header=False, encoding='utf-8-sig')
        print(f"Noise 데이터 저장 완료: {len(noise_results)}개")
        noise_results = []
    else:
        print("Noise 결과가 없습니다. noise.csv를 저장하지 않습니다.")

    print(f"{end_idx}/{total_rows}개 데이터 처리 완료. 중간 결과를 저장했습니다.")

print("모든 데이터 처리가 완료되었습니다.")

#### 2. 특수문자 비율을 계산해 재분류할 행들을 필터링
- 1에서 생성된 clean.csv, noise.csv를 살펴보니, 특수문자 비율 0.3을 기준으로 오분류된 데이터가 많음
- -> filtered.csv에 저장한 뒤 재분류

In [ ]:
def calculate_special_char_ratio(text):
    if len(text) == 0:
        return 0
    special_chars = re.findall(r'[^가-힣\s…·.%美北中朴日靑∼↑英與↓]', str(text)) # 한글, 공백, test data에서 많이 쓰인 상위 15개 문자는 특수문자 비율 계산에서 제외
    return round((len(special_chars) / len(text)), 3)

clean = pd.read_csv('1_clean.csv')
noise = pd.read_csv('1_noise.csv')

clean['ratio'] = clean['text'].apply(calculate_special_char_ratio)
noise['ratio'] = noise['text'].apply(calculate_special_char_ratio)
clean_filtered = clean[clean['ratio'] > 0.3]
noise_filtered = noise[noise['ratio'] < 0.3]

filtered = pd.concat([clean_filtered, noise_filtered])
filtered.to_csv('2_filtered.csv', index=False)

#### 3. LLM으로 2차 분류
- 2에서 생성된 filtered.csv를 재분류
1. filtered.csv를 보니 특수문자 비율이 0.13 미만인 행은 전부 clean함 -> 'natural'로 라벨링하고 filtered_clean에 저장
2. 나머지는 LLM으로 재분류
- -> filtered_clean, filtered_noise, filtered_labeled.csv 생성

In [ ]:
"""
filtered.csv 파일의 데이터를 gemini 1.5 flash LLM을 사용하여 재분류하고,
is_natural 라벨을 추가하여 filtered_labeled.csv에 저장하는 코드입니다.
"""

### SETTING ###
API_KEY = ''
INPUT_FILE = '2_filtered.csv'
###############

# API 키 설정
os.environ["GOOGLE_API_KEY"] = API_KEY

# 데이터 로드
data_file = INPUT_FILE
try:
    data = pd.read_csv(data_file)
    print(f"총 {len(data)}개의 데이터를 로드했습니다.")
except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다: {data_file}")
    exit(1)
except Exception as e:
    print(f"데이터 로드 중 오류 발생: {e}")
    exit(1)

# is_natural 열 추가 및 초기화
data['is_natural'] = None

# ratio가 natural_threshold 미만인 행은 'natural'으로 라벨링
natural_threshold = 0.13
data.loc[data['ratio'] < natural_threshold, 'is_natural'] = 'natural'

# ratio가 natural_threshold 미만인 행들을 별도로 3_filtered_clean.csv에 저장
clean_initial = data[data['ratio'] < natural_threshold][['ID', 'text', 'target']]
clean_initial.to_csv('3_filtered_clean.csv', index=False, encoding='utf-8-sig')
print(f"ratio가 {natural_threshold} 미만인 {len(clean_initial)}개의 행을 3_filtered_clean.csv에 저장했습니다.")

# ratio가 natural_threshold 이상인 행들만 재분류 대상
reclassify_data = data[data['ratio'] >= natural_threshold].copy()
print(f"재분류 대상 데이터 수: {len(reclassify_data)}개")

if reclassify_data.empty:
    print("재분류할 데이터가 없습니다.")
else:
    # 모델 초기화
    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

    # 프롬프트 템플릿 정의
    prompt_template = PromptTemplate(
        input_variables=["batch"],
        template="""
        다음은 뉴스 제목 형식의 텍스트 10개입니다:

        {batch}

        각 텍스트가 자연스러운지 부자연스러운지 판단해 주세요. 
        자연스러운 문장은 문법적으로 올바르고, 특수문자나 무작위 문자의 삽입 없이 의미가 명확하게 전달되는 문장입니다.
        부자연스러운 문장은 단어 중간에 특수문자, 알파벳, 숫자가 무작위로 삽입되어 있거나 단어가 왜곡되어 문맥 파악이 어려운 문장입니다.
        반드시 각 번호에 해당하는 텍스트의 분류 결과를 'natural' 또는 'unnatural'으로만 대답해 주세요.
        응답은 번호 순서대로 한 줄에 하나씩 작성해 주세요.
        """
    )

    # 결과를 저장할 리스트 초기화
    filtered_clean_results = []
    filtered_noise_results = []

    # 배치 크기 설정
    batch_size = 10
    total_rows = len(reclassify_data)

    # 출력 파일명 설정
    filtered_clean_output_file = '3_filtered_clean.csv'
    filtered_noise_output_file = '3_filtered_noise.csv'

    # 기존 결과 파일이 존재하면 삭제하여 새로 생성 (clean_initial은 이미 저장했으므로 여기서는 삭제하지 않음)
    if os.path.exists(filtered_noise_output_file):
        os.remove(filtered_noise_output_file)
        print(f"{filtered_noise_output_file} 파일을 삭제하고 새로 생성합니다.")

    # 배치 처리
    for start_idx in range(0, total_rows, batch_size):
        end_idx = min(start_idx + batch_size, total_rows)
        batch = reclassify_data.iloc[start_idx:end_idx]

        # Prepare batch text
        batch_texts = []
        batch_ids = []
        for i, (index, row) in enumerate(batch.iterrows(), 1):
            id = row['ID']
            text = row['text']
            batch_texts.append(f"{i}. ID: {id}, Text: {text}")
            batch_ids.append(id)

        batch_prompt = "\n".join(batch_texts)

        # 프롬프트 생성
        prompt = prompt_template.format(batch=batch_prompt)

        success = False
        retry_count = 0
        max_retries = 5

        while not success and retry_count < max_retries:
            try:
                # 모델 예측
                response = model.invoke(prompt)

                # AIMessage 객체에서 실제 텍스트 추출
                response_text = response.content.strip()
                classifications = response_text.split('\n')

                if len(classifications) != len(batch_ids):
                    raise ValueError("응답된 분류 개수가 입력된 텍스트 개수와 일치하지 않습니다.")

                for idx, classification in enumerate(classifications):
                    id = batch_ids[idx]
                    text = batch.iloc[idx]['text']
                    target = batch.iloc[idx]['target']

                    # 번호와 점을 제거하여 실제 분류 결과 추출
                    match = re.match(r'\d+\.\s*(.+)', classification)
                    if match:
                        classification = match.group(1).strip()
                    else:
                        classification = classification.strip()

                    if classification.lower() == 'natural':
                        label = 'natural'
                        filtered_clean_results.append({
                            "ID": id,
                            "text": text,
                            "target": target,
                            "label": label
                        })
                    elif classification.lower() == 'unnatural':
                        label = 'unnatural'
                        filtered_noise_results.append({
                            "ID": id,
                            "text": text,
                            "target": target,
                            "label": label
                        })
                    else:
                        # 예기치 않은 응답은 'unnatural'로 처리
                        label = 'unnatural'
                        filtered_noise_results.append({
                            "ID": id,
                            "text": text,
                            "target": target,
                            "label": label
                        })

                    # is_natural 컬럼 업데이트
                    data.loc[data['ID'] == id, 'is_natural'] = label

                    print(f"ID {id} 처리 완료: {label}")

                success = True

                # 요청 사이에 지연 시간 추가
                time.sleep(2)  # 지연 시간을 늘려서 API 부하를 줄입니다

            except Exception as e:
                retry_count += 1
                print(f"배치 {start_idx + 1}-{end_idx} 처리 중 오류 발생: {e}")
                if 'ResourceExhausted' in str(e):
                    wait_time = 30  # 쿼터 초과 시 30초 대기
                    print(f"{wait_time}초 후에 재시도합니다.")
                    time.sleep(wait_time)
                else:
                    wait_time = 10  # 기타 오류 시 10초 대기
                    print(f"{wait_time}초 후에 재시도합니다.")
                    time.sleep(wait_time)

        if not success:
            print(f"배치 {start_idx + 1}-{end_idx} 처리를 건너뜁니다. 'unnatural'으로 라벨링합니다.")
            for idx in range(len(batch)):
                id = batch.iloc[idx]['ID']
                text = batch.iloc[idx]['text']
                target = batch.iloc[idx]['target']
                label = 'unnatural'
                filtered_noise_results.append({
                    "ID": id,
                    "text": text,
                    "target": target,
                    "label": label
                })
                # is_natural 컬럼 업데이트
                data.loc[data['ID'] == id, 'is_natural'] = label
                print(f"ID {id} 처리를 실패하여 'unnatural'으로 라벨링했습니다.")

        # 배치 처리 후 결과 저장
        # filtered_clean.csv 저장 (이미 초기 clean을 저장했으므로 append)
        if filtered_clean_results:
            filtered_clean_df = pd.DataFrame(filtered_clean_results)
            # 헤더는 첫 번째 배치에서만 추가
            header = not os.path.exists(filtered_clean_output_file)
            filtered_clean_df.to_csv(filtered_clean_output_file, mode='a', index=False, header=header, encoding='utf-8-sig')
            print(f"Filtered Clean 데이터 저장 완료: {len(filtered_clean_results)}개")
            filtered_clean_results = []  # 리스트 초기화
        else:
            print("Filtered Clean 결과가 없습니다. filtered_clean.csv를 저장하지 않습니다.")

        # filtered_noise.csv 저장
        if filtered_noise_results:
            filtered_noise_df = pd.DataFrame(filtered_noise_results)
            # 헤더는 첫 번째 배치에서만 추가
            header = not os.path.exists(filtered_noise_output_file)
            filtered_noise_df.to_csv(filtered_noise_output_file, mode='a', index=False, header=header, encoding='utf-8-sig')
            print(f"Filtered Noise 데이터 저장 완료: {len(filtered_noise_results)}개")
            filtered_noise_results = []  # 리스트 초기화
        else:
            print("Filtered Noise 결과가 없습니다. filtered_noise.csv를 저장하지 않습니다.")

        print(f"{end_idx}/{total_rows}개 데이터 처리 완료. 중간 결과를 저장했습니다.")

    # 최종 결과 저장
    # 필요한 열만 선택하여 filtered_labeled.csv로 저장
    final_df = data[['ID', 'text', 'target', 'is_natural']].copy()
    final_df.rename(columns={'is_natural': 'label'}, inplace=True)
    final_df.to_csv('3_filtered_labeled.csv', index=False, encoding='utf-8-sig')
    print("filtered_labeled.csv 파일이 성공적으로 저장되었습니다.")


#### 4. 2차 분류한 결과를 반영
- -> clean_again.csv, noise_again.csv 생성

In [ ]:
"""
clean.csv와 noise.csv 파일을 불러와 filtered_labeled.csv의 라벨에 따라 데이터를 재분류하고,
최종적으로 clean_again.csv와 noise_again.csv 파일로 저장하는 스크립트입니다.
"""
# 파일 경로 설정
clean_file = '1_clean.csv'
noise_file = '1_noise.csv'
filtered_labeled_file = '3_filtered_labeled.csv'
clean_again_file = '4_clean_again.csv'
noise_again_file = '4_noise_again.csv'

# 데이터 로드
try:
    clean_df = pd.read_csv(clean_file)
    print(f"'{clean_file}' 파일을 성공적으로 로드했습니다. 총 {len(clean_df)}개의 행.")
except FileNotFoundError:
    print(f"'{clean_file}' 파일을 찾을 수 없습니다. 빈 DataFrame을 생성합니다.")
    clean_df = pd.DataFrame(columns=['ID', 'text', 'target'])

try:
    noise_df = pd.read_csv(noise_file)
    print(f"'{noise_file}' 파일을 성공적으로 로드했습니다. 총 {len(noise_df)}개의 행.")
except FileNotFoundError:
    print(f"'{noise_file}' 파일을 찾을 수 없습니다. 빈 DataFrame을 생성합니다.")
    noise_df = pd.DataFrame(columns=['ID', 'text', 'target'])

try:
    filtered_labeled_df = pd.read_csv(filtered_labeled_file)
    print(f"'{filtered_labeled_file}' 파일을 성공적으로 로드했습니다. 총 {len(filtered_labeled_df)}개의 행.")
except FileNotFoundError:
    print(f"'{filtered_labeled_file}' 파일을 찾을 수 없습니다. 스크립트를 종료합니다.")
    exit(1)

# 'natural' 라벨인 데이터 처리
natural_df = filtered_labeled_df[filtered_labeled_df['label'] == 'natural']
print(f"'natural' 라벨인 데이터 수: {len(natural_df)}개.")

# 'natural' 라벨인 데이터 중 noise.csv에 있는 행 찾기
natural_in_noise = natural_df[natural_df['ID'].isin(noise_df['ID'])]
print(f"'natural' 라벨인 데이터 중 noise.csv에 있는 행 수: {len(natural_in_noise)}개.")

# noise.csv에서 해당 행 삭제
if not natural_in_noise.empty:
    noise_df = noise_df[~noise_df['ID'].isin(natural_in_noise['ID'])]
    print(f"noise.csv에서 {len(natural_in_noise)}개의 행을 삭제했습니다.")

    # clean.csv에 해당 행 추가 (중복 방지)
    clean_df = pd.concat([clean_df, natural_in_noise[['ID', 'text', 'target']]], ignore_index=True)
    clean_df.drop_duplicates(subset=['ID'], inplace=True)
    print(f"clean.csv에 {len(natural_in_noise)}개의 행을 추가했습니다.")
else:
    print("재분류할 'natural' 라벨의 행이 noise.csv에 존재하지 않습니다.")

# 'unnatural' 라벨인 데이터 처리
unnatural_df = filtered_labeled_df[filtered_labeled_df['label'] == 'unnatural']
print(f"'unnatural' 라벨인 데이터 수: {len(unnatural_df)}개.")

# 'unnatural' 라벨인 데이터 중 clean.csv에 있는 행 찾기
unnatural_in_clean = unnatural_df[unnatural_df['ID'].isin(clean_df['ID'])]
print(f"'unnatural' 라벨인 데이터 중 clean.csv에 있는 행 수: {len(unnatural_in_clean)}개.")

# clean.csv에서 해당 행 삭제
if not unnatural_in_clean.empty:
    clean_df = clean_df[~clean_df['ID'].isin(unnatural_in_clean['ID'])]
    print(f"clean.csv에서 {len(unnatural_in_clean)}개의 행을 삭제했습니다.")

    # noise.csv에 해당 행 추가 (중복 방지)
    noise_df = pd.concat([noise_df, unnatural_in_clean[['ID', 'text', 'target']]], ignore_index=True)
    noise_df.drop_duplicates(subset=['ID'], inplace=True)
    print(f"noise.csv에 {len(unnatural_in_clean)}개의 행을 추가했습니다.")
else:
    print("재분류할 'unnatural' 라벨의 행이 clean.csv에 존재하지 않습니다.")

# 최종 DataFrame 저장
# clean_again.csv 저장
clean_again_df = clean_df[['ID', 'text', 'target']]
clean_again_df.to_csv(clean_again_file, index=False, encoding='utf-8-sig')
print(f"최종 clean_again.csv 파일을 저장했습니다. 총 {len(clean_again_df)}개의 행.")

# noise_again.csv 저장
noise_again_df = noise_df[['ID', 'text', 'target']]
noise_again_df.to_csv(noise_again_file, index=False, encoding='utf-8-sig')
print(f"최종 noise_again.csv 파일을 저장했습니다. 총 {len(noise_again_df)}개의 행.")

#### 5. LLM으로 3차 분류
- clean_again.csv에 오분류된 데이터를 보니 대부분 특수문자 비율이 0.13~0.29임. -> LLM으로 한번 더 분류
- -> 최종 분류물인 label_noise, text_noise 생성

In [ ]:
"""
4_clean_again.csv의 text의 ratio가 0.13 초과 0.29 미만인 행을 추출해서
gemini로 자연스러운지 부자연스러운지 여부를 검사한 뒤
자연스러운 데이터는 기존의 4_clean_again.csv와 합쳐 label_noise.csv로 저장하고, (ID, text, target으로 구성)
부자연스러운 데이터는 기존의 4_noise_again.csv와 합쳐 text_noise.csv로 저장합니다. (ID, text, target으로 구성)
저장할 때 label_noise.csv, text_noise.csv 각각의 전체 데이터 수를 출력합니다.
"""

# API key 설정
API_KEY = ''
os.environ["GOOGLE_API_KEY"] = API_KEY

# 파일 경로 설정
clean_again_file = '4_clean_again.csv'
noise_again_file = '4_noise_again.csv'
label_noise_file = 'split_train_data/label_noise.csv'
text_noise_file = 'split_train_data/text_noise.csv'

# 데이터 로드
try:
    clean_df = pd.read_csv(clean_again_file)
    print(f"'{clean_again_file}' 파일을 성공적으로 로드했습니다. 총 {len(clean_df)}개의 행.")
except FileNotFoundError:
    print(f"'{clean_again_file}' 파일을 찾을 수 없습니다. 스크립트를 종료합니다.")
    exit(1)

try:
    noise_df = pd.read_csv(noise_again_file)
    print(f"'{noise_again_file}' 파일을 성공적으로 로드했습니다. 총 {len(noise_df)}개의 행.")
except FileNotFoundError:
    print(f"'{noise_again_file}' 파일을 찾을 수 없습니다. 스크립트를 종료합니다.")
    exit(1)

# ratio 계산 함수 정의
def calculate_special_char_ratio(text):
    # 텍스트 길이가 0인 경우를 처리
    if len(text) == 0:
        return 0
    # 특수문자 비율 계산
    special_chars = re.findall(r'[^가-힣\s…·.%美北中朴日靑∼↑英與↓]', str(text))  # 한글과 공백, 일부 한자 및 기호를 제외한 모든 문자
    return round((len(special_chars) / len(text)), 3)

# ratio 계산
clean_df['ratio'] = clean_df['text'].apply(calculate_special_char_ratio)

# 조건에 맞는 행 추출
subset_df = clean_df[(clean_df['ratio'] > 0.13) & (clean_df['ratio'] < 0.29)]
print(f"검사 대상 데이터 수: {len(subset_df)}개")

if subset_df.empty:
    print("검사 대상 데이터가 없습니다. 기존 데이터를 그대로 저장합니다.")
    # 기존 데이터를 그대로 저장
    clean_df[['ID', 'text', 'target']].to_csv(label_noise_file, index=False, encoding='utf-8-sig')
    noise_df.to_csv(text_noise_file, index=False, encoding='utf-8-sig')
    print(f"최종 label_noise.csv 파일을 저장했습니다. 총 {len(clean_df)}개의 행.")
    print(f"최종 text_noise.csv 파일을 저장했습니다. 총 {len(noise_df)}개의 행.")
else:
    # Gemini 모델 초기화
    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

    # 프롬프트 템플릿 정의
    prompt_template = PromptTemplate(
        input_variables=["batch"],
        template="""
        다음은 뉴스 제목 형식의 텍스트 10개입니다:

        {batch}

        각 텍스트가 자연스러운지 부자연스러운지 판단해 주세요. 
        자연스러운 문장은 문법적으로 올바르고, 특수문자나 무작위 문자의 삽입 없이 의미가 명확하게 전달되는 문장입니다.
        부자연스러운 문장은 단어 중간에 특수문자, 알파벳, 숫자가 무작위로 삽입되어 있거나 단어가 왜곡되어 문맥 파악이 어려운 문장입니다.
        반드시 각 번호에 해당하는 텍스트의 분류 결과를 'natural' 또는 'unnatural'으로만 대답해 주세요.
        응답은 번호 순서대로 한 줄에 하나씩 작성해 주세요.
        """
    )

    # 검사 대상 데이터로부터 필요한 컬럼 추출
    classify_df = subset_df[['ID', 'text', 'target']].copy()

    # 결과를 저장할 리스트 초기화
    natural_results = []
    unnatural_results = []

    # 배치 크기 설정
    batch_size = 10
    total_rows = len(classify_df)

    # 배치 처리
    for start_idx in range(0, total_rows, batch_size):
        end_idx = min(start_idx + batch_size, total_rows)
        batch = classify_df.iloc[start_idx:end_idx]

        # Prepare batch text
        batch_texts = []
        batch_ids = []
        for i, (index, row) in enumerate(batch.iterrows(), 1):
            id = row['ID']
            text = row['text']
            batch_texts.append(f"{i}. ID: {id}, Text: {text}")
            batch_ids.append(id)

        batch_prompt = "\n".join(batch_texts)

        # 프롬프트 생성
        prompt = prompt_template.format(batch=batch_prompt)

        success = False
        retry_count = 0
        max_retries = 5

        while not success and retry_count < max_retries:
            try:
                # 모델 예측
                response = model.invoke(prompt)

                # AIMessage 객체에서 실제 텍스트 추출
                response_text = response.content.strip()
                classifications = response_text.split('\n')

                if len(classifications) != len(batch_ids):
                    raise ValueError("응답된 분류 개수가 입력된 텍스트 개수와 일치하지 않습니다.")

                for idx, classification in enumerate(classifications):
                    id = batch_ids[idx]
                    text = batch.iloc[idx]['text']
                    target = batch.iloc[idx]['target']

                    # 번호와 점을 제거하여 실제 분류 결과 추출
                    match = re.match(r'\d+\.\s*(.+)', classification)
                    if match:
                        classification = match.group(1).strip()
                    else:
                        classification = classification.strip()

                    if classification.lower() == 'natural':
                        natural_results.append({
                            "ID": id,
                            "text": text,
                            "target": target
                        })
                    elif classification.lower() == 'unnatural':
                        unnatural_results.append({
                            "ID": id,
                            "text": text,
                            "target": target
                        })
                    else:
                        # 예기치 않은 응답은 'unnatural'로 처리
                        unnatural_results.append({
                            "ID": id,
                            "text": text,
                            "target": target
                        })

                    print(f"ID {id} 처리 완료: {classification}")

                success = True

                # 요청 사이에 지연 시간 추가
                time.sleep(2)  # 지연 시간을 늘려서 API 부하를 줄입니다

            except Exception as e:
                retry_count += 1
                print(f"배치 {start_idx + 1}-{end_idx} 처리 중 오류 발생: {e}")
                if 'ResourceExhausted' in str(e):
                    wait_time = 30  # 쿼터 초과 시 30초 대기
                    print(f"{wait_time}초 후에 재시도합니다.")
                    time.sleep(wait_time)
                else:
                    wait_time = 10  # 기타 오류 시 10초 대기
                    print(f"{wait_time}초 후에 재시도합니다.")
                    time.sleep(wait_time)

        if not success:
            print(f"배치 {start_idx + 1}-{end_idx} 처리를 건너뜁니다. 'unnatural'로 처리합니다.")
            for idx in range(len(batch)):
                id = batch_ids[idx]
                text = batch.iloc[idx]['text']
                target = batch.iloc[idx]['target']
                unnatural_results.append({
                    "ID": id,
                    "text": text,
                    "target": target
                })
                print(f"ID {id} 처리를 실패하여 'unnatural'로 처리했습니다.")

    # 검사 대상에서 제외된 clean_df의 나머지 데이터 추출
    remaining_clean_df = clean_df[~clean_df['ID'].isin(subset_df['ID'])][['ID', 'text', 'target']]

    # 자연스러운 데이터 합치기
    natural_df = pd.DataFrame(natural_results)
    final_clean_df = pd.concat([remaining_clean_df, natural_df], ignore_index=True)
    final_clean_df.drop_duplicates(subset=['ID'], inplace=True)

    # 부자연스러운 데이터 noise_df에 추가
    unnatural_df = pd.DataFrame(unnatural_results)
    final_noise_df = pd.concat([noise_df, unnatural_df], ignore_index=True)
    final_noise_df.drop_duplicates(subset=['ID'], inplace=True)

    # 최종 데이터 저장
    final_clean_df.to_csv(label_noise_file, index=False, encoding='utf-8-sig')
    final_noise_df.to_csv(text_noise_file, index=False, encoding='utf-8-sig')

    print(f"최종 label_noise.csv 파일을 저장했습니다. 총 {len(final_clean_df)}개의 행.")
    print(f"최종 text_noise.csv 파일을 저장했습니다. 총 {len(final_noise_df)}개의 행.")


#### 6. split 과정 중 생긴 파일들 삭제

In [ ]:
# 현재 디렉토리 내에 있는 모든 .csv 파일 찾기
csv_files = glob.glob("*.csv")

# 각 .csv 파일 삭제
for file in csv_files:
    try:
        os.remove(file)
        print(f"삭제: {file}")
    except Exception as e:
        print(f"삭제 오류 {file}: {e}")
